In [1]:
!echo -e "pyngrok==7.2.0\nfastapi==0.115.0\ntogether==1.3.1\nuvicorn==0.31.1\npydantic==2.9.2" > requirements.txt
!cat requirements.txt

pyngrok==7.2.0
fastapi==0.115.0
together==1.3.1
uvicorn==0.31.1
pydantic==2.9.2


In [3]:
%%capture --no-stderr
%pip install -r requirements.txt -qU
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4
%pip install -qU pypdf
%pip install langchain-huggingface -qU

In [14]:
%pip install -qU langchain-groq
%pip install python-multipart
%pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.6 MB/s eta 0:00:00


In [5]:
from pyngrok import ngrok
from google.colab import userdata

# Replace 'YOUR_NGROK_AUTHTOKEN' with the authtoken you copied from the ngrok dashboard
authtoken = userdata.get("NGROK_KEY")
ngrok.set_auth_token(authtoken)

In [8]:
# Note: You can use other API such as Groq, MistralAI, OpenAI, Google, and so on. In this example I use Groq
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

··········


In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
# model_kwargs = {'device': 'cuda:0'}
# embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs=model_kwargs,)
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

NameError: name '_C' is not defined

In [15]:
import getpass
import os

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [17]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

In [9]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

In [26]:
vector_store = InMemoryVectorStore.from_documents(documents=[],embedding=embeddings_model)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [27]:
hehe = conversational_rag_chain.invoke(
    {"input": "Detail of RAG?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

In [6]:
from google.colab import userdata
import os
from fastapi import APIRouter, Path, Body, HTTPException, Query, status, Depends, UploadFile, File
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from together import Together
from pyngrok import ngrok
import uvicorn
import nest_asyncio  # Add this to allow nested event loops in Colab or Jupyter
from pydantic import BaseModel
from enum import Enum
from typing import Annotated, List
import logging
# Apply the nest_asyncio patch to allow running nested event loops
nest_asyncio.apply()

In [22]:
from langchain_community.document_loaders import PyPDFLoader
import shutil
UPLOAD_DIR = "./uploaded_files/"
async def save_file(file: UploadFile) -> str:
    os.makedirs(UPLOAD_DIR, exist_ok=True)
    file_path = os.path.join(UPLOAD_DIR, file.filename)
    with open(file_path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)
    return file_path
async def load_pdf_files(vector_store, file_path):
    loader = PyPDFLoader(file_path)
    pages = []
    async for page in loader.alazy_load():
        pages.append(page)
    vector_store.add_documents(pages)

In [23]:
class ChatRole(str, Enum):
    USER = "user"
    SYSTEM = "system"
    ASSISTANT = "assistant"


class Message(BaseModel):
    role: ChatRole
    content: str
    model_config = {
        "json_schema_extra": {
            "role": "user",
            "content": "Hello, how are you",
        }
    }

In [29]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import uvicorn

# Initialize FastAPI app
app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Adjust the allowed origins as needed
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/chat")  # FastAPI route for POST
def chat(message: Annotated[
        Message,
        Body(
            example={
                "role": "user",
                "content": "Hello, how are you",
            }
        ),
    ]):
    res = conversational_rag_chain.invoke(
        {"input": message.content},
        config={
            "configurable": {"session_id": "abc123"}
        },  # constructs a key "abc123" in `store`.
    )
    return Message(role=ChatRole.ASSISTANT, content=res['answer'])

@app.post("/uploadfiles/")
async def create_upload_files(files: list[UploadFile]):
    for file in files:
        try:
            # Check if the uploaded file is a PDF
            if file.content_type != "application/pdf":
                raise HTTPException(status_code=400, detail="Invalid file type. Only PDF files are allowed.")

            # Save the uploaded file
            file_path = await save_file(file)

            # Load PDF pages into the vector store
            await load_pdf_files(vector_store=vector_store, file_path=file_path)

        except HTTPException as e:
            # Reraise HTTP exceptions (e.g., invalid file type)
            raise e
        except Exception as e:
            # Log the error for debugging purposes
            logging.error(f"Error during PDF upload and processing: {e}")

            # Return a general error message
            raise HTTPException(status_code=500, detail="An error occurred while processing the PDF.")
    return {"message": "Files uploaded and processed successfully."}
if __name__ == '__main__':
    # Start ngrok
    url = ngrok.connect(8000)  # Port 8000 for FastAPI
    print(f" * ngrok tunnel: {url}")

    # Start Uvicorn server using Server class to avoid asyncio issues in Colab
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    server.run()

 * ngrok tunnel: NgrokTunnel: "https://3864-34-138-224-120.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [341]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     58.186.92.194:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     58.186.92.194:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     58.186.92.194:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     58.186.92.194:0 - "POST /uploadfiles HTTP/1.1" 307 Temporary Redirect
INFO:     58.186.92.194:0 - "POST /uploadfiles/ HTTP/1.1" 200 OK
INFO:     58.186.92.194:0 - "POST /uploadfiles HTTP/1.1" 307 Temporary Redirect
INFO:     58.186.92.194:0 - "POST /uploadfiles/ HTTP/1.1" 200 OK
INFO:     58.186.92.194:0 - "OPTIONS /chat HTTP/1.1" 200 OK
INFO:     58.186.92.194:0 - "POST /chat HTTP/1.1" 422 Unprocessable Entity
INFO:     58.186.92.194:0 - "POST /uploadfiles HTTP/1.1" 307 Temporary Redirect
INFO:     58.186.92.194:0 - "POST /uploadfiles/ HTTP/1.1" 200 OK
INFO:     58.186.92.194:0 - "POST /chat HTTP/1.1" 422 Unprocessable Entity
INFO:     58.186.92.194:0 - "POST /chat HTTP/1.1" 422 Unprocessable Entity
INFO:     58.186.92.194:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     58.186.92.194:0 - "POST /chat HTTP/1

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [341]


KeyboardInterrupt: 